<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Model" data-toc-modified-id="Model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Model</a></span><ul class="toc-item"><li><span><a href="#Language-Modeling" data-toc-modified-id="Language-Modeling-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Language Modeling</a></span></li><li><span><a href="#N-grams" data-toc-modified-id="N-grams-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>N-grams</a></span><ul class="toc-item"><li><span><a href="#Unigrams" data-toc-modified-id="Unigrams-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Unigrams</a></span></li><li><span><a href="#Bigrams" data-toc-modified-id="Bigrams-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Bigrams</a></span></li><li><span><a href="#Trigrams" data-toc-modified-id="Trigrams-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Trigrams</a></span></li></ul></li><li><span><a href="#Trigrams-with-linear-interpolation" data-toc-modified-id="Trigrams-with-linear-interpolation-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Trigrams with linear interpolation</a></span><ul class="toc-item"><li><span><a href="#Trigrams" data-toc-modified-id="Trigrams-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Trigrams</a></span></li><li><span><a href="#Trigrams-with-linear-interpolation" data-toc-modified-id="Trigrams-with-linear-interpolation-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Trigrams with linear interpolation</a></span></li></ul></li></ul></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementation</a></span></li><li><span><a href="#WikiText-103-dataset-processing" data-toc-modified-id="WikiText-103-dataset-processing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>WikiText 103 dataset processing</a></span></li><li><span><a href="#WikiText-103-Benchmark" data-toc-modified-id="WikiText-103-Benchmark-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>WikiText 103 Benchmark</a></span></li></ul></div>

## Model

### Language Modeling

Un modelo de lenguaje busca calcular la probabilidad de observar una secuancia de tokens o palabras $w_{0}, \ldots, w_{N}$, usando la regla de la cadena de probabilidades la probabilidad conjunta de observar tal secuencia se descompone en la siguiente expresión
:
\begin{equation*}
p(w_{0}, \ldots, w_{N}) = \prod_{i=0}^{N}p(w_{i}|\ldots, w_{i-1})
\end{equation*}

### N-grams 

Un modelo de N-grams posee una fuerte suposición sobre las probabilidades condicionales de una palabra y su historia ($p(w_{i}|\ldots, w_{i-1})$), así la probabilidad de obserbar la palabra $w_{i}$ depende de los N-1 palabras anteriores. Por ejemplo:

#### Unigrams
\begin{equation*}
p(w_{i}|\ldots, w_{i-1}) = p(w_{i})
\end{equation*}

#### Bigrams
\begin{equation*}
p(w_{i}|\ldots, w_{i-1}) = p(w_{i}| w_{i-1})
\end{equation*}

#### Trigrams
\begin{equation*}
p(w_{i}|\ldots, w_{i-1}) = p(w_{i}| w_{i-2}, w_{i-1})
\end{equation*}

### Trigrams with linear interpolation

El baseline a implementar consiste de un modelo de trigramas con interpolación lineal para evitar el problema de **zero-count**, este problema ocurre cuando la probabilidad de observar un trigrama es cero, lo que trae por consecuencia que la probabilidad conjunta sea cero, pues usando regla de la cadena y markov de segundo orden la probabilidad conjunta se descompone en el producto de los trigramas de la secuencia y basta que un trigrama tenga probabilidad cero para hechar a perder el cálculo de la probabilidad.

#### Trigrams
\begin{equation*}
p(w_{0}, \ldots, w_{N}) = \prod_{i=0}^{N} p(w_{i}| w_{i-2}, w_{i-1})
\end{equation*}

#### Trigrams with linear interpolation

La probabilidad que entrega este modelo es la combinación convexa de las probabilidades que entregan los modelos unigrams, bigrams y trigrams.

\begin{align}
\nonumber
& p(w_{0}, \ldots, w_{N}) = \prod_{i=0}^{N}\lambda_{1} p(w_{i}| w_{i-2}, w_{i-1})+\lambda_{2} p(w_{i}| w_{i-1})+\lambda_{3} p(w_{i})\\ \nonumber
& \lambda_{i}\geq 0, \; \lambda_{1}+\lambda_{2}+\lambda_{3}=1\ \nonumber
\end{align}

Los parámetros del modelo se estiman contando casos favorables versus totales, por ejemplo la probabilidad estimada de un trigrama viene dada por la siguiente expresión:
\begin{equation*}
q(w_{i}|w_{i-2}, w_{i}) = \frac{Count(w_{i-2}, w_{i-1}, w_{i})}{Count(w_{i-2}, w_{i-1})}
\end{equation*}

## Implementation 

In [1]:
%%capture
import nltk
nltk.download('reuters', quiet=True)
nltk.download('punkt', quiet=True)

In [2]:
import numpy as np
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import defaultdict

In [3]:
class Trigrams():     
    def fit(self, corpus):
        """
        Ajusta el modelo en el corpus de entrenamiento, guarda los parámetros en atributos.

        Parameters:
            corpus: {list of str}, shape (corpus_size) 
            corpus de entrenamiento tokenizado.
        Returns:
            self: object
        """
        ## Crear diccionarios que guardaran la probabilidad de cada n-gram
        model1 = defaultdict(lambda: 0)
        model2 = defaultdict(lambda: defaultdict(lambda: 0))
        model3 = defaultdict(lambda: defaultdict(lambda: 0))

        ## Contar frecuencia de co-ocurrencia
        N = len(corpus)
        for sentence in corpus:
            # Unigrams
            model1[None]=N
            for w1 in sentence:
                model1[w1]+=1
            # Bigrams
            model2[None][None]=N
            for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
                model2[w1][w2] += 1
            # Trigrams
            for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
                model3[(w1, w2)][w3] += 1

        ## Transformar conteo a probabilidades
        # Trigrams
        for w1_w2 in model3:
            w1, w2 = w1_w2[0], w1_w2[1]
            total_count = model2[w1][w2]
            for w3 in model3[w1_w2]:
                model3[w1_w2][w3] /= total_count
        # Bigrams        
        for w1 in model2:
            total_count = model1[w1]
            for w2 in model2[w1]:
                model2[w1][w2] /= total_count
        # Unigrams    
        total_count = sum(model1.values())
        for w1 in model1:
            model1[w1] /= total_count

        self.model1 = model1
        self.model2 = model2
        self.model3 = model3     
        
    def predict_proba_trigam(self, lamb, trigram):
        """
        Calcula la probabilidad de un trigrama usando interpolación lineal.
            - p(w3|w1,w2) = lamb_1*q(w3|w1,w2)+lamb_2*q(w3|w2)+lamb_3*q(w3)
            - lamb>=0, lamb_1+lamb_2+lamb_3=1
        Parameters:
            lamb: {array}, shape =3
            hiperparámetros del modelo, cada componente debe ser positivo y deben sumar 1.

        Returns:
            score: float
            probabilidad del trigrama
        """
        w1, w2, w3 = trigram
        proba = lamb[0]*self.model3[(w1,w2)][w3]+lamb[1]*self.model2[w2][w3]+lamb[2]*self.model1[w3]
        return proba
        
    def get_perplexity(self, lamb, corpus):

        """
        Obtiene la perplexity del modelo en un conjunto de validación
            -perplexity = exp(NLL/N), donde NLL es la negative-log-likelihood del modelo y 
             N el número de trigramas en el corpus.

        Parameters:
            - corpus: {list of str}, shape (corpus_size) 
              corpus de validación tokenizado.
            - lamb: {array}, shape =3
              hiperparámetros del modelo, cada componente debe ser positivo y deben sumar 1.
        Returns:
            score: float
            perplexity
        """

        N = 0 #contador de trigramas del conjunto de validación
        NLL = 0 #negative-log-likelihood
        for sentence in corpus:
            for trigram in trigrams(sentence, pad_right=True, pad_left=True):
                trigram_proba = self.predict_proba_trigam(lamb, trigram)
                NLL = NLL - np.log(trigram_proba) 
                N = N+1
        perplexity = np.exp(NLL/N)

        return perplexity
    

In [4]:
corpus = reuters.sents()
print(corpus[0])
print(len(corpus))

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.']
54716


In [5]:
model = Trigrams()
%time model.fit(corpus)

Wall time: 14.8 s


In [6]:
lamb = [0.7, 0.2, 0.1]
%time model.get_perplexity(lamb, corpus)

Wall time: 14.9 s


8.383050732528654

In [7]:
#Calcular probabilidad de un trigrama
example = trigrams(corpus[0], pad_right=True, pad_left=True)
example_trigrams = []
for trigram in example:
    example_trigrams.append(trigram)
    
print(example_trigrams[2])
model.predict_proba_trigam(lamb, example_trigrams[2])

('ASIAN', 'EXPORTERS', 'FEAR')


0.7043479387228446

## WikiText 103 dataset processing

As the files were already tokenize we must build the tokens array for each dataset. Using the spaces left by the tokenization and the file names the three arrays of tokens are built.

In [8]:
import os
import re
import shutil

from nltk.tokenize import word_tokenize

shutil.unpack_archive('../wikitext-103-v1.zip', extract_dir='dataset')
working_dir = os.path.join(os.getcwd(), 'dataset', 'wikitext-103')

wikitext_files = os.listdir(working_dir)

wiki_train = []
wiki_train_vocabulary = []
wiki_test = []
wiki_valid = []

for wikitext_file in wikitext_files:
    with open(os.path.join(working_dir, wikitext_file), encoding='utf-8') as data_file:
        for index, line in enumerate(data_file):
            # Filter out empty lines and headers
            if len(line) < 3 or line[1] == '=':
                continue
            
            else:
                if re.match(r'.*\.train\..*', wikitext_file):
                    wiki_train.append(line.strip().split(' '))
                    wiki_train_vocabulary.append(line.strip())
                
                elif re.match(r'.*\.test\..*', wikitext_file):
                    wiki_test.append(line.strip().split(' '))
                    
                elif re.match(r'.*\.valid\..*', wikitext_file):
                    wiki_valid.append(line.strip().split(' '))
                    
shutil.rmtree('dataset')
                    
print(f'Paragraphs on train:      {len(wiki_train)}')
print(f'Paragraphs on test:       {len(wiki_test)}')
print(f'Paragraphs on validation: {len(wiki_valid)}')

Paragraphs on train:      859532
Paragraphs on test:       2183
Paragraphs on validation: 1841


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

def get_vocabulary(dataset):
    vocabulary = []

    vectorizer = CountVectorizer()
    
    vectorizer.fit_transform(dataset)
    vectorizer._validate_vocabulary()
    
    vocabulary = vectorizer.get_feature_names()

    return vocabulary


def remove_out_of_vocabulary(vocabulary, dataset):
    new_dataset = []
    deleted = 0
    
    vocabulary = set(vocabulary)
    
    for paragraph in dataset:
        new_paragraph = []
        inter = set(paragraph) & vocabulary
        
        for word in paragraph:
            if word in inter:
                new_paragraph.append(word)
            
            else:
                deleted += 1

        new_dataset.append(new_paragraph)

    print(f'{deleted} words were out of vocabulary')
    return new_dataset

%time vocabulary = get_vocabulary(wiki_train_vocabulary)

print(f'Vocabulary has {len(vocabulary)} tokens')

%time wiki_test = remove_out_of_vocabulary(vocabulary, wiki_test)
%time wiki_valid = remove_out_of_vocabulary(vocabulary, wiki_valid)

Wall time: 1min 25s
Vocabulary has 225785 tokens
79926 words were out of vocabulary
Wall time: 124 ms
72275 words were out of vocabulary
Wall time: 116 ms


## WikiText 103 Benchmark

Now we must train a new model. This process is similar to the one shown in section 2.

In [10]:
wikitext_model = Trigrams()
%time wikitext_model.fit(wiki_train)

Wall time: 7min 54s


In [ ]:
from numpy import arange

results = {}

for lambda1 in arange(0, 1, 0.1):
    for lambda2 in arange(0, 1, 0.1):
        for lambda3 in arange(0, 1, 0.1):
            wikitext_lamb = [
                round(lambda1, 1), 
                round(lambda2, 1), 
                round(lambda3, 1)
            ]
            
            if sum(wikitext_lamb) != 1.0:
                continue
            
            results[str(wikitext_lamb)] = wikitext_model.get_perplexity(wikitext_lamb, wiki_test)

In [ ]:
import pandas

def results_as_table():
    rows = []

    for key, value in results.items():
        row = {
            'configuration': key,
            'perplexity': value
        }

        rows.append(row)

    df = pandas.DataFrame(rows)
    
    return df

results_as_table()

We take the top 5 perplexity configurations and we run them with the validation dataset.

In [ ]:
df = results_as_table()
best_configurations = df.sort_values('perplexity', ascending=True).head()
best_configurations

In [ ]:
import ast

validation_configurations = best_configurations['configuration'].tolist()
validation_configurations = list(map(lambda config : ast.literal_eval(config), validation_configurations))

results = {}

for config in validation_configurations:
    results[str(config)] = wikitext_model.get_perplexity(config, wiki_valid)
    
results_as_table()